In [94]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from fastapi.encoders import jsonable_encoder

In [197]:
OPEN_API_KEY = "YOUR_API_KEY"
BASE_URL = "https://openai.vocareum.com/v1"
GPT_MODEL = "gpt-4o-mini-2024-07-18"

In [188]:
llm = ChatOpenAI(model=GPT_MODEL, temperature=0, base_url = BASE_URL, api_key = OPEN_API_KEY)

task = "Generate twenty diverse real estate listings."
sample_listing = '''
Sample Listing : 

Neighborhood: Green Oaks
Price: $800000
Bedrooms: 3
Bathrooms: 2
House Size: 2000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
'''


In [189]:
class RealEstateListing(BaseModel):
    neighborhood: str = Field(description="The name of the neighborhood where the property is located")
    price: NonNegativeInt = Field(description="Price of the property in USD")
    bedrooms: NonNegativeInt = Field(description="Number of bedrooms in the property")
    bathrooms: NonNegativeInt = Field(description="Number of bathrooms in the property")
    house_size: NonNegativeInt = Field(description="Size of the property in square feet")
    description: str = Field(description="Description of the property")
    neighborhood_description: str = Field(description="Description of the neighborhood")

class ListingsCollection(BaseModel):
    listings: List[RealEstateListing] = Field(description="List of Real Estate Listings")

parser = PydanticOutputParser(pydantic_object=ListingsCollection)

In [190]:
prompt = PromptTemplate(
    template="{task}\n{sample_listing}\n\n{format_instructions}",
    input_variables=["task, sample_listing"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)
query = prompt.format(task=task, sample_listing=sample_listing)
print(query)

Generate twenty diverse real estate listings.

Sample Listing : 

Neighborhood: Green Oaks
Price: $800000
Bedrooms: 3
Bathrooms: 2
House Size: 2000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation an

In [191]:
response = llm(query)

In [195]:
result = parser.parse(response.content)
df = pd.DataFrame(jsonable_encoder(result.listings))
df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Green Oaks,800000,3,2,2000,Welcome to this eco-friendly oasis nestled in ...,"Green Oaks is a close-knit, environmentally-co..."
1,Downtown Heights,1200000,2,2,1500,Experience urban living at its finest in this ...,Downtown Heights is a vibrant urban area fille...
2,Sunnyvale,650000,4,3,2500,"This spacious 4-bedroom, 3-bathroom home in Su...",Sunnyvale is known for its family-friendly atm...
3,Lakeside,950000,5,4,3200,Discover luxury living in this stunning 5-bedr...,Lakeside is a picturesque community known for ...
4,Maplewood,450000,3,2,1800,"Charming 3-bedroom, 2-bathroom home in Maplewo...","Maplewood is a quiet, suburban neighborhood wi..."


In [196]:
df.to_csv('listings.csv',index_label = 'id')